In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import gradio as gr

# 3. Sample Data (Minimal dataset for demonstration)
data = {
    'text': [
        "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005.", # Spam
        "WINNER!! As a valued network customer you have been selected to receivea £900 prize reward!", # Spam
        "Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free!", # Spam
        "Congrats! You have won a free vacation. Click here", # Spam
        "URGENT! Your account needs immediate attention. Click link.", # Spam
        "Nah I don't think he goes to usf, he lives around here though", # Ham
        "Even my brother is not like to speak with me. They treat me like aids patent.", # Ham
        "I'm going to try for 2 months ha ha only joking", # Ham
        "Ok lar... Joking wif u oni...", # Ham
        "Hi, how are you doing today?", # Ham
        "Meeting scheduled for 3 PM tomorrow.", # Ham
        "Can you please send me the report?", # Ham
    ],
    'label': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0] # 1 for Spam, 0 for Not Spam (Ham)
}
df = pd.DataFrame(data)

# 4. Create and Train the Model Pipeline
# Uses CountVectorizer to convert text to word counts
# Uses Multinomial Naive Bayes classifier
model = make_pipeline(CountVectorizer(), MultinomialNB())
model.fit(df['text'], df['label'])

# 5. Define the prediction function for Gradio
def predict_spam(text):
    """
    Predicts if the input text is spam or not spam.
    Args:
        text (str): The input text message.
    Returns:
        str: Prediction result ("Spam" or "Not Spam").
    """
    prediction = model.predict([text]) # Predict expects an iterable
    if prediction[0] == 1:
        return "🚨 Spam Detected!"
    else:
        return "✅ Not Spam (Ham)."

# 6. Create the Gradio interface
iface_spam = gr.Interface(
    fn=predict_spam,
    inputs=gr.Textbox(lines=5, label="Enter Text Message", placeholder="Type your message here..."),
    outputs=gr.Textbox(label="Prediction Result"),
    title="📧 Spam Detection Classifier",
    description="Enter a text message to classify it as Spam or Not Spam using a Naive Bayes model.",
    examples=[
        ["Free prize! Click now!"],
        ["Hi Mom, running late for dinner."],
        ["Your order has shipped."]
    ]
)

# 7. Launch the interface
print("Launching Spam Detection Gradio Interface...")
iface_spam.launch(share=True) # share=True provides a public

In [ ]:
import gradio as gr
# Removed 'Conversation' as it's not directly importable this way and wasn't used in the function below
from transformers import pipeline
import torch # Ensure torch is available

# 3. Load a pre-trained conversational model pipeline
# Using a smaller model suitable for Colab environments
# You might need to accept terms or log in to Hugging Face for some models
print("Loading text-generation model...")
try:
    # Using DialoGPT-medium as an example.
    # *** CHANGED task from "conversational" to "text-generation" ***
    chatbot_pipeline = pipeline("text-generation", model="microsoft/DialoGPT-medium", pad_token_id=50256) # Added pad_token_id for open-ended generation
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have accepted model terms on Hugging Face if required, or try a different model.")
    # Fallback or exit if model loading fails
    chatbot_pipeline = None # Set to None if loading failed

# 4. Define the prediction function for Gradio ChatInterface
# Note: The simple approach below relies on the pipeline's internal state
# and basic text generation. It might not be truly conversational without
# explicitly managing history input to the model.

def chatbot_response(message, history):
    """
    Generates a response from the chatbot model using text-generation pipeline.
    Args:
        message (str): The user's input message.
        history (list): A list of previous user/bot message pairs (provided by Gradio, format depends on chatbot type).
    Returns:
        str: The chatbot's generated response.
    """
    if chatbot_pipeline is None:
        return "Error: Chatbot model could not be loaded."

    # Simple approach for text-generation pipeline:
    # Pass the message, generate text, and try to extract the response part.
    # This is basic and might include the prompt; better methods involve chat templates or direct model generation.
    try:
        # Generate text based on the input message
        # max_length can be adjusted; setting it slightly longer than input helps get a response
        responses = chatbot_pipeline(message, max_length=100, num_return_sequences=1)

        if responses and len(responses) > 0 and 'generated_text' in responses[0]:
             generated_text = responses[0]['generated_text']
             # Try to extract only the newly generated part (heuristic)
             # This assumes the response starts with the input message, which is common but not guaranteed.
             if generated_text.startswith(message):
                 # Remove the input prompt and potential leading/trailing spaces
                 bot_message = generated_text[len(message):].strip()
                 # Handle cases where only the prompt was returned or empty response
                 if not bot_message:
                      bot_message = "..." # Or some default response
             else:
                 # If the prompt isn't there, return the whole generation (might be okay)
                 bot_message = generated_text.strip()

             # Avoid returning empty strings
             if not bot_message:
                bot_message = "I'm not sure how to respond to that."

        else:
            print("Pipeline did not return expected output format.")
            bot_message = "Sorry, I couldn't generate a response."


    except Exception as e:
        print(f"Error during chatbot inference: {e}")
        bot_message = "Sorry, I encountered an error processing your message."

    return bot_message

# 5. Create and launch the Gradio Chat Interface
print("Launching Chatbot Gradio Interface...")
iface_chat = gr.ChatInterface(
    fn=chatbot_response,
    title="💬 Simple LLM Chatbot (DialoGPT)",
    description="Chat with a conversational AI model (using text-generation). Type your message and press Enter.",
    examples=[["Hello!"], ["How does photosynthesis work?"], ["Tell me a joke."]],
    # *** ADDED type='messages' to gr.Chatbot to address UserWarning ***
    chatbot=gr.Chatbot(height=400, type='messages'), # Customize chatbot display height
    textbox=gr.Textbox(placeholder="Ask me anything...", container=False, scale=7),
    # *** REMOVED retry_btn argument as it caused TypeError ***
    # *** REMOVED undo_btn and clear_btn arguments to simplify the call ***
    # undo_btn="Delete Previous", # This argument seems valid but removed for testing
    # clear_btn="Clear Chat",    # This argument seems valid but removed for testing
)

iface_chat.launch(share=True) # share=True provides a public link



In [ ]:
import os
import gradio as gr
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
# Import Google AI components
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import google.generativeai as genai # Import google-generativeai for configuration

# --- Access Google AI API Key from Colab Secrets ---
from google.colab import userdata

try:
    # *** LOOKING FOR GOOGLE_API_KEY now ***
    google_api_key = userdata.get('GOOGLE_API_KEY')
    # Configure the google-generativeai library
    genai.configure(api_key=google_api_key)
    print("Google AI API Key loaded and configured successfully.")
except userdata.SecretNotFoundError:
    print("GOOGLE_API_KEY secret not found. Please add it in Colab Secrets.")
    print("Get a key from https://aistudio.google.com/app/apikey")
    google_api_key = None
except Exception as e:
    print(f"An error occurred accessing secrets or configuring Google AI: {e}")
    google_api_key = None
# ----------------------------------------------------

# 3. Prepare Document Data (Example about Chennai)
# Create a dummy text file in Colab's environment
chennai_text = """
Chennai, formerly known as Madras, is the capital city of the Indian state of Tamil Nadu.
Located on the Coromandel Coast off the Bay of Bengal, it is one of the largest cultural, economic and educational centres of south India.
Chennai is known as the "Detroit of India" for its automobile industry. It is also a major centre for music, art and culture.
The city is home to historical landmarks like Fort St. George, built in 1644, and several ancient temples.
Marina Beach in Chennai is one of the longest urban beaches in the world.
The climate is typically hot and humid for most of the year.
Key areas include Mylapore, T. Nagar, Adyar, and George Town.
"""
with open("chennai_info.txt", "w") as f:
    f.write(chennai_text)

# 4. Setup Langchain Components (only if key is available)
retrieval_chain = None # Initialize retrieval_chain to None
if google_api_key:
    print("Setting up Langchain components with Google AI...")
    try:
        # Load Document
        loader = TextLoader("./chennai_info.txt")
        docs = loader.load()

        # Split Document
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        split_docs = text_splitter.split_documents(docs)

        # Create Embeddings (Using Google's embedding model)
        print("Loading Google AI embedding model...")
        # *** USING GoogleGenerativeAIEmbeddings ***
        embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

        # Create Vector Store
        print("Creating vector store...")
        vector_store = FAISS.from_documents(split_docs, embedding_model)

        # Create Retriever
        retriever = vector_store.as_retriever(search_kwargs={"k": 2}) # Retrieve top 2 relevant chunks

        # Define LLM (Using Google's Gemini Pro model via Langchain)
        print("Setting up Google AI LLM (Gemini)...")
        # *** USING ChatGoogleGenerativeAI ***
        llm = ChatGoogleGenerativeAI(model="gemini-pro",
                                     temperature=0.7, # Adjust creativity
                                     convert_system_message_to_human=True) # Helps with some prompt structures

        # Define Prompt Template
        template = """
        Answer the user's question based only on the following context:
        Context:
        {context}

        Question: {input}
        Answer:
        """
        prompt = ChatPromptTemplate.from_template(template)

        # Create Chains
        document_chain = create_stuff_documents_chain(llm, prompt)
        retrieval_chain = create_retrieval_chain(retriever, document_chain)
        print("Langchain setup complete.")

    except Exception as e:
        print(f"Error during Langchain setup with Google AI: {e}")
        retrieval_chain = None # Ensure chain is None if setup fails

else:
    print("Skipping Langchain setup as GOOGLE_API_KEY is missing.")
    # retrieval_chain remains None

# 5. Define the Q&A function for Gradio
def answer_question(question):
    """
    Answers a question based on the loaded document using Langchain.
    Args:
        question (str): The user's question.
    Returns:
        str: The answer generated by the Langchain chain or an error message.
    """
    if retrieval_chain is None:
        # Updated error message
        return "Error: Langchain components not initialized. Please ensure GOOGLE_API_KEY secret is set and valid."
    if not question:
        return "Please ask a question."

    try:
        print(f"Invoking chain with question: {question}")
        response = retrieval_chain.invoke({"input": question})
        print(f"Chain response: {response}")
        # The actual answer is usually in the 'answer' key of the response dictionary
        return response.get("answer", "Sorry, I couldn't generate an answer based on the context.")
    except Exception as e:
        # Catch potential API errors from Google AI as well
        print(f"Error during Langchain invocation: {e}")
        error_message = f"An error occurred: {e}"
        # Provide more specific feedback if it's a known API issue (optional)
        if "API key not valid" in str(e):
             error_message = "Error: The provided Google AI API Key is not valid. Please check it in Colab Secrets."
        elif "quota" in str(e).lower():
             error_message = "Error: You may have exceeded your Google AI API quota."
        return error_message

# 6. Create the Gradio interface
print("Launching Langchain Q&A Gradio Interface (using Google AI)...")
iface_langchain_google = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, label="Ask a Question about Chennai", placeholder="e.g., What is Chennai known for?"),
    outputs=gr.Textbox(label="Answer"),
    title="📚 Langchain Q&A about Chennai (Google AI/Gemini)",
    description="Ask questions based on a small text about Chennai. Uses RAG with Google Gemini. Requires GOOGLE_API_KEY.",
    examples=[
        ["What is Marina Beach?"],
        ["Which industries is Chennai known for?"],
        ["Where is Fort St. George located?"]
    ]
)

# 7. Launch the interface
iface_langchain_google.launch(share=True)

